In [1]:
# change working directory to root directory
import os

os.chdir("../")
os.getcwd()

'/Users/mac/Desktop/msda_labour_graph/MSDA jobs graph '

## Overview
This notebook will be a sandbox to test the integration of new skillNer modules and source for the package migration 

In [2]:
## only run when semsi update their data hhhhhhh ! (it takes a lot of time trust me )
"""
from skillNer.fetch import fetch_skills_list
# load skill list 
response = fetch_skills_list()
import re
skills_dict = {}
for i,skill in enumerate(response) : 
    print(i/len(response),skill)
    print('-------------')
    # get info 
    skill_id = skill['id']
    skill_name = skill['name']
    skill_type = skill['type']['name']
    # clean definitions and textify 
    cleaned_skill_name = re.sub(r"\([^()]*\)", "", skill_name)
    skill_obj = Text(cleaned_skill_name)
    # add to dict 
    skills_dict[skill_id] = {'skill_id':skill_id,
                             'skill_name' :skill_name , 
                             'skill_type':skill_type,
                             'skill_obj':skill_obj }
# store ready to use skill dict
final_skill_dict = {}
for skill_key in skills_dict.keys() :
            skill_name = skills_dict[skill_key]['skill_name']
            skill_obj = skills_dict[skill_key]['skill_obj']
            final_skill_dict[skill_key] = {'skill_name' :skill_name , 
                                     'skill_cleaned':skill_obj.transformed_text,
                                     'skill_type':skill_type,
                                     'skill_lemmed':skill_obj.lemmed(),
                                     'skill_stemmed':skill_obj.stemmed(),
                                     'skill_len' : len(skill_obj)
                                   }
            
import json
with open('./skillNer/skills_processed.json', 'w', encoding='utf-8') as f:
    json.dump(final_skill_dict, f, ensure_ascii=False, indent=4)
"""
print()

In [3]:
# load libs 
import pprint
from skillNer.text_class import Text,nlp

import spacy
from spacy.matcher import PhraseMatcher
#nlp = spacy.load("en_core_web_lg")
# load stop_words to be filtred for n_gram (should be decripted in future)
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [4]:
# load libs 
import pprint
from skillNer.text_class import Text

import spacy
from spacy.matcher import PhraseMatcher
nlp = spacy.load("en_core_web_lg")
# load stop_words to be filtred for n_gram (should be decripted in future)
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [5]:
# import processed skills db 
import json

with open('./skillNer/skills_processed.json') as json_file:
    skills_db = json.load(json_file)

In [6]:
skills_db['ES7947C5DF69201A2341']

{'skill_name': 'Airworthiness Engineering',
 'skill_cleaned': 'airworthiness engineering',
 'skill_type': 'Hard Skill',
 'skill_lemmed': 'airworthiness engineering',
 'skill_stemmed': 'airworthi engin',
 'skill_len': 2}

In [7]:
## dist used in n_gram_process should be stored as skills_db since its fixed !
sgrams = [skills_db[key]['skill_stemmed'] for key in skills_db if skills_db[key]['skill_len']==2]
def get_dist(array):
    words = []
    for val in array :
        vals = val.split(' ')
        for v in vals :
            
            words.append(v)
    
    import collections
    
    a = words
    counter=collections.Counter(a)
    counter = dict(counter)
   # print(counter)
    max_ = max(list(counter.values()))
    min_ = min(list(counter.values()))

    if max_==min_ :
        ret = counter
    else :
        ret  = {k: 1-((v-min_)/(max_-min_)) for k, v in counter.items()}
    return ret
sgrams_skills_tokens_dist =  get_dist(sgrams)

In [18]:

# load matchers 
def load_matchers(include=['full_matcher','ngram_matcher','uni_gram_matcher']):
    # creators 
    def get_full_matcher():
        # init matcher 
        full_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
        # populate matcher
        for key in skills_db :
            # get skill info 
            skill_id =key
            skill_full_name = skills_db[key]['skill_cleaned']
            skill_len = skills_db[key]['skill_len']
            if skill_len>1 :
                # add to matcher
                skill_full_name_spacy = nlp.make_doc(skill_full_name)
                full_matcher.add(str(skill_id), [skill_full_name_spacy])
                
        return full_matcher
    def get_ngram_matcher():
        ## init matcher
        ngram_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
        # populate matcher
        for key in skills_db :
            # get skill info 
            skill_id =key
            skill_lemmed = skills_db[key]['skill_lemmed']
            skill_lemmed_tokens = [w for w in skill_lemmed.split(' ') 
                                    if not(w in stop_words or w.isdigit())]
            
            skill_len = skills_db[key]['skill_len']
            if skill_len >1 :## add only ngram skills 
                # add full_stemed to matcher
                skill_lemmed_spacy = nlp.make_doc(skill_lemmed)
                ngram_matcher.add(str(skill_id), [skill_lemmed_spacy])
                ## add tokens to matcher 
                for token in skill_lemmed_tokens :
                    ## give id that ref 1_gram matching
                    id_ = skill_id+'_1w'
                    ngram_matcher.add(str(id_), [nlp.make_doc(token)])
                
        return ngram_matcher
    def get_uni_gram__matcher():
        # init matcher 
        single_gram_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
        # populate matcher
        for key in skills_db :
            # get skill info 
            skill_id =key
            skill_stemmed = skills_db[key]['skill_stemmed']
            skill_len = skills_db[key]['skill_len']
            if skill_len==1 :
                # add to matcher
                skill_stemmed_spacy = nlp.make_doc(skill_stemmed)
                single_gram_matcher.add(str(skill_id), [skill_stemmed_spacy])
                
        return single_gram_matcher    
    
    load_matcher = {
        'full_matcher' :get_full_matcher,
        'ngram_matcher' : get_ngram_matcher,
        'uni_gram_matcher' : get_uni_gram__matcher,
    }
    
    # load to return 
    loader = {}
    for type_ in include :
        print('loading ',type_,"...")
        loader[type_] = load_matcher[type_]()
    return loader
        

In [9]:
def get_full_match_skills(text_obj,matcher):
    skills = []
    doc = nlp(text_obj.transformed_text)
    
    for match_id, start, end in matcher(doc):
        id_ = matcher.vocab.strings[match_id]
        ## add full_match to store
        skills.append({'skill_id':id_,
                       'doc_node_value':str(doc[start:end]),
                       'doc_node_id':list(range(start,end))})
        ## mutate text tokens metadata (unmatch attr) 
        for token in text_obj[start:end] :
            token.is_matchable = False
        
    return skills,text_obj

def get_sub_match_skills(text_obj,matcher):
    skills_full = []
    skills = []
    sub_matches= []
    full_matches = []
    doc = nlp(text_obj.lemmed())
    for match_id, start, end in matcher(doc):
        id_ = matcher.vocab.strings[match_id]
        if '_1w' in id_ : 
            sub_matches.append((id_,match_id, start, end))
        else : 
            full_matches.append((id_,match_id, start, end))
            
            
    for match in  full_matches : 
            id_,match_id, start, end = match
            ## full matches no need for scoring
            ## check if any intersection betwenn full matcher and sub matcher (priority to full)
            is_matchable = [1 for token in  text_obj[start:end] if token.is_matchable]
            if len(is_matchable)!= 0 :
                
                 skills_full.append({'skill_id':id_,
                                      'doc_node_value':str(doc[start:end]),
                                      'doc_node_id':list(range(start,end))})
     
                 ## mutate text tokens metadata (unmatch attr) - only in full match stemmed (100% confident )
                 for token in text_obj[start:end] :
                      token.is_matchable = False  
                    
            
    for match in sub_matches:
            id_,match_id, start, end = match
            # add unigram macthes only if not matched in parent modules or not stop word
            if text_obj[start].is_matchable and (not text_obj[start].is_stop_word):
                skills.append({'skill_id':id_,
                               'doc_node_value':str(doc[start:end]),
                               'doc_node_id':start})
                
    return skills_full,skills,text_obj

def get_single_match_skills(text_obj,matcher):
    skills = []
   
    doc = nlp(text_obj.stemmed())
    for match_id, start, end in matcher(doc):
        id_ = matcher.vocab.strings[match_id]
        if text_obj[start].is_matchable : 
            skills.append({'skill_id':id_,
                           'doc_node_value':str(doc[start:end]),
                           'doc_node_id':start})

    return skills

In [62]:
# process_n_gram ,process_uni_gram : filter and score  
## utils 
import numpy as np
def one_gram_sim(text_str,skill_str):
    # transform into sentence 
    text = text_str+' '+skill_str
    tokens = nlp(text)
    token1, token2 = tokens[0], tokens[1]
   # print(token1, token2)
    return token1.similarity(token2)
def similarity(texts):
    doc1 = nlp(texts[0])
    doc2 = nlp(texts[1])
 

    return  doc1.similarity(doc2)
def get_sim(skill_name, f ,input_text):
    text = skill_name.lower().replace(f,'').strip()
    #print('props : ', skill_name,'/', f ,'/',input_text)
    #print('sim /',text,'/',input_text)
    return similarity([text,input_text])

def make_one(cluster,len_):
    a = [1]*len_
    return [1*(i in cluster) for i,one in enumerate(a) ]
def split_at_values(lst, val):
    return [i for i, x in enumerate(lst) if x != val]
        
def grouper(iterable,dist):
    prev = None
    group = []
    for item in iterable:
        if  prev==None or item - prev <= dist:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item
    if group:
        yield group 
def get_clusters (co_oc):
    clusters = []
    for i,row in enumerate(co_oc):
        clusts = list(grouper(split_at_values(row, 0),1))
        if clusts != [] :
            #print(i,[c for c in clusts if i in c][0])
            a = [c for c in clusts if i in c][0]
            if a not in clusters :
                
                clusters.append(a)
    return clusters
def is_low_frequency(match_str,skill_id):
    skill_name = skills_db[skill_id]['skill_stemmed'].split(' ')
  
    if sgrams_skills_tokens_dist[skill_name[0]]>= sgrams_skills_tokens_dist[skill_name[1]]:
      
        return skill_name[0]==match_str
    else : 
        return skill_name[1]==match_str
def retain(text , tokens,skill_id,sk_look,corpus ) :
    ## get id 
    real_id = sk_look[skill_id].split('_1w')[0]
    ## get len 
    len_ = skills_db[real_id]['skill_len']
    len_condition = corpus[skill_id].dot(tokens)/len_
    
    s_gr = np.array(list(tokens))*np.array(list(corpus[skill_id]))
    def condition(x): return x == 1
    s_gr_ind = [idx for idx, element in enumerate(s_gr) if condition(element)][0]
    #print('len',len_,real_id,s_gr_ind )
    if len_condition >=0.5 : 
        #print('debug',debug[real_id] ,corpus[skill_id] ,tokens , len_condition  )
        if len_>2   :
                #print('ngram', True , real_id)
                score = len_condition
                return (True , {'skill_id':real_id,
                                'doc_node_id':  [i for i,val in enumerate(s_gr) if val==1],
                                'score':round(score,2)
                               })
            
        if  is_low_frequency(text[s_gr_ind],real_id)   :
                #print('2gram', look_up_ngram[real_id],"/", text[s_gr_ind] ,"/",' '.join(text))
                score = get_sim(skills_db[real_id]['skill_lemmed'], text[s_gr_ind] ,' '.join(text))
                return (True , {'skill_id':real_id,
                                'score':round(score,2),
                                 'doc_node_id': [i for i,val in enumerate(s_gr) if val==1]
                               })
        return (False , '')

    else :
        return (False , '')
    
def get_corpus(text,matches):
    len_ = len(text)
    corpus = []
    look_up = {}
    unique_skills = list(set([match['skill_id'] for match in matches]))
    skill_text_match_bin = [0]*len_
    for index,skill_id in enumerate(unique_skills) : 
        
        on_inds = [match['doc_node_id'] for match in matches if match['skill_id']==skill_id]
        skill_text_match_bin_updated = [(i in on_inds)*1 for i,_ in enumerate(skill_text_match_bin)]
        corpus.append(skill_text_match_bin_updated)
        look_up[index] = skill_id
                   
    return np.array(corpus) , look_up
    
## main functions
def process_n_gram(matches,text_obj):
                if len(matches)==0:
                     return matches
                
                # get text spans with  conflict 
                text_tokens = text_obj.lemmed().split(' ')
                len_ = len(text_tokens)
                ## create corpus matrix  
                corpus,look_up = get_corpus(text_tokens,matches)
                ## generate spans 
                co_occ = np.matmul(corpus.T, corpus) # co-occurence of tokens aij : co-occurence of token i with token j
                clusters = get_clusters(co_occ)
                ones = [ make_one(cluster,len_) for cluster in clusters]
                spans = [(np.array(one),np.array([a_[0] for a_ in np.argwhere(corpus.dot(one)!=0)])) 
                         for one in ones]
                # filter and score 
                new_spans = []
                for span in spans :
                    tokens,skill_ids = span 
                    new_skill_obj = []
                    for sk_id in skill_ids : 
                        retain_ , r_sk_id = retain(text_tokens , tokens,sk_id,look_up,corpus )
                        if   retain_ :
                            new_skill_obj.append(r_sk_id)
                    # get max for each span 
                    scores = [sk['score'] for sk in new_skill_obj]
                    if scores!=[]:
                        max_score_index = np.array(scores).argmax() 
                
                        new_spans.append(new_skill_obj[max_score_index])                
                
                return new_spans
            
def process_unigram(matches,text_obj):
                original_text = text_obj.transformed_text.split(' ')                
                res = []
                for match in matches :
                    id_ = match['skill_id']
                    match_id = match['doc_node_id']

                    skill_str = skills_db[id_]['skill_cleaned']
                    #print(match_id)
                    text_str = original_text[match_id]
                    sim = one_gram_sim(skill_str,text_str)
                    #match['doc_node_id'] = [match['doc_node_id']] # for normalisation purpose

                        
                    res.append({'skill_id':id_,
                                'doc_node_id':[match_id],
                                'doc_node_value':match['doc_node_value'],
                                'score':round(sim,2)})  
                return res

# Skills extractor test  :

In [30]:
matchers = load_matchers()

loading  full_matcher ...
loading  ngram_matcher ...
loading  uni_gram_matcher ...


In [72]:
## get matches 
input_ = 'esport management '
text = Text(input_)

skills_full,text = get_full_match_skills(text,matchers['full_matcher'])
skills_sub_full,skills_ngram,text = get_sub_match_skills(text,matchers['ngram_matcher'])
skills_uni = get_single_match_skills(text,matchers['uni_gram_matcher'])

In [73]:
## process 
n_gram_pro = process_n_gram(skills_ngram,text)
uni_gram_pro = process_unigram(skills_uni,text)

In [74]:
## results 
pprint.pprint({'full_match':skills_full,
               'ngram_full_match':skills_sub_full,
               'ngram_scored':n_gram_pro,
               'unigram_scored':uni_gram_pro
              })

{'full_match': [],
 'ngram_full_match': [{'doc_node_id': [0, 1],
                       'doc_node_value': 'esport management',
                       'skill_id': 'ES8AAD06BE8119038221'}],
 'ngram_scored': [],
 'unigram_scored': []}


In [68]:
skills_db['KS1208C6D8DGHVYQTSLR']

{'skill_name': 'Advanced Disaster Management',
 'skill_cleaned': 'advanced disaster management',
 'skill_type': 'Hard Skill',
 'skill_lemmed': 'advanced disaster management',
 'skill_stemmed': 'advanc disast manag',
 'skill_len': 3}